In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'd:\\MLOPS\\End-To-End-MlOps-Project'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = os.getenv('MLFLOW_TRACKING_URI')
os.environ['MLFLOW_TRACKING_USERNAME'] = os.getenv('MLFLOW_TRACKING_USERNAME')
os.environ['MLFLOW_TRACKING_PASSWORD'] = os.getenv('MLFLOW_TRACKING_PASSWORD')

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [20]:
@dataclass
class ModelEvalConfig:
   dir: Path
   test_arr:Path
   all_params: str
   model: Path
   metrics: str


In [14]:
from src.dsproject.utils.utils import create_dir,read_yaml,save_json,load_obj
from src.dsproject.constants.yaml_path import *
from src.dsproject import logging

In [53]:
class ConfigManager:
    def __init__(self,config_file_path=Config_file_path,prams_file_path=Param_file_path,schema_file_path=Schema_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(prams_file_path)
        self.schema=read_yaml(schema_file_path)

        create_dir([self.config.artifacts_root])
    def get_model_eval_config(self):
        try:
         config=self.config.Model_Eval
         create_dir([config.dir])
         schema=self.schema
         params=self.params.RandomForestClassifier
         model_eval_config=ModelEvalConfig(
                  dir=config.dir,
                  test_arr=config.test_arr,
                  all_params=params,
                  model=config.model,
                  metrics=config.metrics
               )
         return model_eval_config

         
        except Exception as e:
         logging.info(f'error in MODEL eval config {str(e)}')
         raise e
        

In [54]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
import pickle
from mlflow.models import infer_signature

In [57]:
class ModelEval:
    def __init__(self, config:ModelEvalConfig):
        self.config = config
    
    def eval_metrics(self, y_actual, y_pred):
        try:
            accuracy = accuracy_score(y_actual, y_pred) * 100
            precision = precision_score(y_actual, y_pred) * 100
            recall = recall_score(y_actual, y_pred) * 100
            clf_report=classification_report(y_actual,y_pred)

            return accuracy, precision, recall,clf_report

        except Exception as e:
            logging.info(f'Error in Model evaluation metrics: {str(e)}')
            raise e

    def initiating_model_eval(self):
        try:
            logging.info('Model evaluation started')

            # Load test data and model
            test_arr = np.load(self.config.test_arr,allow_pickle=True)
            model = load_obj(self.config.model)

            x_test = test_arr[:, :-1]
            y_test = test_arr[:, -1]

            # Set MLflow tracking URI
            mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))

            # Infer the signature for the model
            sig = infer_signature(x_test, y_test)

            # Start MLflow run for logging
            with mlflow.start_run():
                # Make predictions
                predict = model.predict(x_test)

                # Calculate evaluation metrics (accuracy, precision, recall)
                accuracy, precision, recall,clf_report = self.eval_metrics(y_actual=y_test, y_pred=predict)
                scores = {'accuracy': accuracy, 'precision': precision, 'recall': recall,"classification report":clf_report}

                # Save metrics as JSON
               
                save_json(filename=self.config.metrics, data=scores)
					 #Log params
                mlflow.log_params(self.config.all_params)
                # Log metrics to MLflow
                mlflow.log_metric('Accuracy', accuracy)
                mlflow.log_metric('Precision', precision)
                mlflow.log_metric('Recall', recall)

                mlflow.log_text(clf_report,'classsification_report.txt')

                # Log the model with signature
                tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
                if tracking_uri_type_store != 'file':
                    mlflow.sklearn.log_model(model, 'model', registered_model_name='best_model')
                else:
                    mlflow.sklearn.log_model(model, 'model', signature=sig)

        except Exception as e:
            logging.info(f'Error in Model evaluation: {e}')
            raise e

In [58]:
try:
            config=ConfigManager()
            model_eval_config=config.get_model_eval_config()
            model_eval=ModelEval(model_eval_config)
            model_eval.initiating_model_eval()

except Exception as e:
            logging.info(f'Error occured {str(e)}')
            raise e

[2024-11-06 18:56:58,750: INFO :utils : Yaml file config\config.yaml created]
[2024-11-06 18:56:58,755: INFO :utils : Yaml file params.yaml created]
[2024-11-06 18:56:58,761: INFO :utils : Yaml file schema.yaml created]
[2024-11-06 18:56:58,764: INFO :utils : created directory at: artifacts]
[2024-11-06 18:56:58,767: INFO :utils : created directory at: artifacts/model_eval]
[2024-11-06 18:56:58,770: INFO :4179958558 : Model evaluation started]


d:\MLOPS\End-To-End-MlOps-Project\ds\Lib\site-packages\_distutils_hack\__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\MLOPS\End-To-End-MlOps-Project\ds\Lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'best_model' already exists. Creating a new version of this model...
2024/11/06 18:57:26 INFO mlflow.store.model_reg